# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. He  ->  J. He  |  ['J. He']
S. Li  ->  S. Li  |  ['S. Li']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 65 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.14684


extracting tarball to tmp_2309.14684...

 done.


J. He  ->  J. He  |  ['J. He']
S. Li  ->  S. Li  |  ['S. Li']


...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2309.14915/sample63.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'juliet_fit_defs.tex' from 'tmp_2309.14915/juliet_fit_defs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'TOI199_param_fit10_commands' from 'tmp_2309.14915/TOI199_param_fit10_commands.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 97 bibliographic references in tmp_2309.14915/sample63.bbl.
Retrieving document from  https://arxiv.org/e-print/2309.14965


extracting tarball to tmp_2309.14965...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.14684-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.14684) | **The ALMA Survey of Star Formation and Evolution in Massive Protoclusters  with Blue Profiles (ASSEMBLE): Core Growth, Cluster Contraction, and  Primordial Mass Segregation**  |
|| F. Xu, et al. -- incl., <mark>J. He</mark>, <mark>S. Li</mark> |
|*Appeared on*| *2023-09-27*|
|*Comments*| *37 pages, 13 figures, 5 tables; accepted for publication in ApJS*|
|**Abstract**| The ALMA Survey of Star Formation and Evolution in Massive Protoclusters with Blue Profiles (ASSEMBLE) aims to investigate the process of mass assembly and its connection to high-mass star formation theories in protoclusters in a dynamic view. We observed 11 massive (Mclump>1000 Msun), luminous (Lbol>10,000 Lsun), and blue-profile (infall signature) clumps by ALMA with resolution of 2200-5500 au at 350 GHz (870 um) in continuum and line emission. 248 dense cores were identified, including 106 cores showing protostellar signatures and 142 prestellar core candidates. Compared to early-stage infrared dark clouds (IRDCs) by ASHES, the core mass and surface density within the ASSEMBLE clumps exhibited significant increment, suggesting concurrent core accretion during the evolution of the clumps. The maximum mass of prestellar cores was found to be 2 times larger than that in IRDCs, indicating evolved protoclusters have the potential to harbor massive prestellar cores. The mass relation between clumps and their most massive core (MMCs) is observed in ASSEMBLE but not in IRDCs, which is suggested to be regulated by multiscale mass accretion. The mass correlation between the core clusters and their MMCs has a steeper slope compared to that observed in stellar clusters, which can be due to fragmentation of the MMC and stellar multiplicity. We observe a decrease in core separation and an increase in central concentration as protoclusters evolve. We confirm primordial mass segregation in the ASSEMBLE protoclusters, possibly resulting from gravitational concentration and/or gas accretion. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.14915-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.14915) | **TOI-199 b: A well-characterized 100-day transiting warm giant planet  with TTVs seen from Antarctica**  |
|| M. J. Hobson, et al. -- incl., <mark>T. Trifonov</mark>, <mark>J. Eberhardt</mark> |
|*Appeared on*| *2023-09-27*|
|*Comments*| *33 pages, 23 figures. Accepted for publication in AJ*|
|**Abstract**| We present the spectroscopic confirmation and precise mass measurement of the warm giant planet TOI-199 b. This planet was first identified in TESS photometry and confirmed using ground-based photometry from ASTEP in Antarctica including a full 6.5$\,$h long transit, PEST, Hazelwood, and LCO; space photometry from NEOSSat; and radial velocities (RVs) from FEROS, HARPS, CORALIE, and CHIRON. Orbiting a late G-type star, TOI-199\,b has a $\mathrm{104.854_{-0.002}^{+0.001} \, d}$ period, a mass of $\mathrm{0.17\pm0.02 \, M_J}$, and a radius of $\mathrm{0.810\pm0.005 \, R_J}$. It is the first warm exo-Saturn with a precisely determined mass and radius. The TESS and ASTEP transits show strong transit timing variations, pointing to the existence of a second planet in the system. The joint analysis of the RVs and TTVs provides a unique solution for the non-transiting companion TOI-199 c, which has a period of $\mathrm{273.69_{-0.22}^{+0.26} \, d}$ and an estimated mass of $\mathrm{0.28_{-0.01}^{+0.02} \, M_J}$. This period places it within the conservative Habitable Zone. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.14965-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.14965) | **A comprehensive forecast for cosmological parameter estimation using  joint observations of gravitational-wave standard sirens and short  $γ$-ray bursts**  |
|| T. Han, S.-J. Jin, J.-F. Zhang, <mark>X. Zhang</mark> |
|*Appeared on*| *2023-09-27*|
|*Comments*| *16 pages, 11 figures*|
|**Abstract**| In the third-generation (3G) gravitational-wave (GW) detector era, the multi-messenger GW observation for binary neutron star (BNS) merger events can exert great impacts on exploring the cosmic expansion history. In this work, we comprehensively explore the potential of 3G GW standard siren observations in cosmological parameter estimations by considering the 3G GW detectors and the future short $\gamma$-ray burst (GRB) detector THESEUS-like telescope joint observations. Based on the 10-year observation of different detection strategies, we predict that the numbers of detectable GW-GRB events are 277-685 with the redshifts $z<4$ and the inclination angles $\iota<17^{\circ}$. For the cosmological analysis, we consider five typical dark energy models, i.e., the $\Lambda$CDM, $w$CDM, $w_0w_a$CDM models, and interacting dark energy (IDE) models (I$\Lambda$CDM and I$w$CDM). We find that GW can tightly constrain the Hubble constant with precisions of $0.09\%$-$0.37\%$, but perform not well in constraining other cosmological parameters. Fortunately, GW could effectively break the cosmological parameter degeneracies generated by the mainstream EM observations, CMB+BAO+SN (CBS). When combining the mock GW data with the CBS data, CBS+GW can tightly constrain the equation of state parameter of dark energy $w$ with a precision of $1.36\%$, close to the standard of precision cosmology. Meanwhile, the addition of GW to CBS could improve constraints on cosmological parameters by $35.3\%$-$92.0\%$. In conclusion, GW standard siren observations from 3G GW detectors could play a crucial role in helping solve the Hubble tension and probe the fundamental nature of dark energy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.14684.md
    + _build/html/tmp_2309.14684/./I14382_contdata.png
    + _build/html/tmp_2309.14684/./I14382_extsources.png
    + _build/html/tmp_2309.14684/./I14498_contdata.png
    + _build/html/tmp_2309.14684/./I14498_extsources.png
exported in  _build/html/2309.14915.md
    + _build/html/tmp_2309.14915/./ground_based_photometry_times.png
    + _build/html/tmp_2309.14915/./TOI-199_rvs_all_fit10.png
    + _build/html/tmp_2309.14915/./TOI-199_ttvs.png
    + _build/html/tmp_2309.14915/./TOI-199_phasefold_1_fit10_bin2.png
    + _build/html/tmp_2309.14915/./TOI-199_phasefold_2_fit10_bin2.png
    + _build/html/tmp_2309.14915/./comparison_lightcurves_zoom_4.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

, and the ASHES Pilot  ([Sanhueza, Contreras and Wu 2019]())  samples are presented in blue, green, and gray colors, respectively. The mean spatial resolution of both the ASSEMBLE and the ASHES surveys are $\sim0.02$ pc, shown with orange shadow. Lower: the 1000 Monte Carlo runs of the probability density distribution of core separation for the ASSEMBLE (blue lines) and the ASHES (gray lines), respectively, considering the Gaussian-like uncertainty of clump distance. The Mann-Whitney U test is performed on each of the sets of core separation distributions, and the distribution of the p-value is shown in the top right. The p-values are much lower than 0.01, showing that two samples share a significantly different distribution of core separation.  (*fig:separation*)

</div>
<div id="div_fig3">

<img src="tmp_2309.14684/./I14382_contdata.png" alt="Fig7.1" width="25%"/><img src="tmp_2309.14684/./I14382_extsources.png" alt="Fig7.2" width="25%"/><img src="tmp_2309.14684/./I14498_contdata.png" alt="Fig7.3" width="25%"/><img src="tmp_2309.14684/./I14498_extsources.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** The ALMA 870 $\mu$m dust continuum emission without primary beam correction as well as extracted cores for two ASSEMBLE clumps (I14382-6017 and I14498-5856). The ALMA mosaicked primary beam responses of 0.5 and 0.2 are outlined by yellow solid and dashed lines respectively. Only the primary beam response of 0.2 is shown on the right panel. The beam size of each continuum image is shown in the bottom left corner. _Left_: the background color map shows the ALMA 870 $\mu$m emission with two colorbars, the first one (grayscale) showing -9 to +9 times the rms noise on a linear scale, then a second one (color-scheme) showing the range +9 times the rms noise to the peak value of the image in an arcsinh stretch. The rms noise and peak intensity are given on the top right. The black contours are from the ATLASGAL 870 $\mu$m continuum emission, with power-law levels that start at $5\sigma$ and end at $I_\mathrm{peak}$, increasing in steps following the power law $f(n)=3\times n^p + 2$ where $n=1,2,3,...N$ and $p$ is determined from $D=3\times N^p+2$($D=I_\mathrm{peak}/\sigma$: the dynamic range; $N=8$: the number of contour levels). The values of each contour level are labeled with a unit of $\jybeam$. _Right_: the background gray-scale map shows the arcsinh-stretch part in the left panel, outlined by the $5\sigma$ contour. The ALMA continuum emission map is smoothed to a circular beam with a size equal to the major axis of the original beam. The cores extracted by $\getsf$ algorithm are presented by red / blue ellipses, as well as black IDs, with numbers in order from North to South. The red and blue ones represent protostellar and prestellar cores defined in Section \ref{result:coreclass}.  (*fig:continuum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.14684"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\periodplanetb}{\ensuremath{104.854_{-0.002}^{+0.001}}}$
$\newcommand{\Kplanetb}{\ensuremath{7.7\pm1.1}}$
$\newcommand{\eccplanetb}{\ensuremath{0.09_{-0.02}^{+0.01}}}$
$\newcommand{\omegaplanetb}{\ensuremath{350_{-4}^{+2}}}$
$\newcommand{\Maplanetb}{\ensuremath{111\pm2}}$
$\newcommand{\massplanetb}{\ensuremath{0.17\pm0.02}}$
$\newcommand{\axisplanetb}{\ensuremath{0.4254 \pm 0.002}}$
$\newcommand{\periodplanetc}{\ensuremath{273.69_{-0.22}^{+0.26}}}$
$\newcommand{\Kplanetc}{\ensuremath{9.1_{-0.4}^{+0.5}}}$
$\newcommand{\eccplanetc}{\ensuremath{0.096_{-0.009}^{+0.008}}}$
$\newcommand{\omegaplanetc}{\ensuremath{8_{-7}^{+4}}}$
$\newcommand{\Maplanetc}{\ensuremath{192_{-3}^{+4}}}$
$\newcommand{\massplanetc}{\ensuremath{0.28_{-0.01}^{+0.02}}}$
$\newcommand{\axisplanetc}{\ensuremath{0.807 \pm 0.003}}$
$\newcommand{\RVoffFEROS}{\ensuremath{51326.7_{-3.1}^{+2.9}}}$
$\newcommand{\RVoffHARPS}{\ensuremath{51350.4_{-0.9}^{+1.0}}}$
$\newcommand{\RVoffCHIRON}{\ensuremath{-0.5_{-3.9}^{+4.2}}}$
$\newcommand{\RVoffCORALIE}{\ensuremath{51331.0_{-3.3}^{+3.1}}}$
$\newcommand{\RVjittFEROS}{\ensuremath{19.9_{-2.1}^{+2.6}}}$
$\newcommand{\RVjittHARPS}{\ensuremath{5.5_{-0.6}^{+0.8}}}$
$\newcommand{\RVjittCHIRON}{\ensuremath{11.9_{-3.7}^{+4.8}}}$
$\newcommand{\RVjittCORALIE}{\ensuremath{5.4_{-3.5}^{+4.4}}}$
$\newcommand{\periodplanetbmaxLn}{\ensuremath{104.855} }$
$\newcommand{\KplanetbmaxLn}{\ensuremath{7.7}}$
$\newcommand{\eccplanetbmaxLn}{\ensuremath{0.102}}$
$\newcommand{\omegaplanetbmaxLn}{\ensuremath{351}}$
$\newcommand{\MaplanetbmaxLn}{\ensuremath{110}}$
$\newcommand{\massplanetbmaxLn}{\ensuremath{0.17}}$
$\newcommand{\axisplanetbmaxLn}{\ensuremath{0.4254}}$
$\newcommand{\periodplanetcmaxLn}{\ensuremath{273.55}}$
$\newcommand{\KplanetcmaxLn}{\ensuremath{8.8}}$
$\newcommand{\eccplanetcmaxLn}{\ensuremath{0.107}}$
$\newcommand{\omegaplanetcmaxLn}{\ensuremath{11}}$
$\newcommand{\MaplanetcmaxLn}{\ensuremath{190}}$
$\newcommand{\massplanetcmaxLn}{\ensuremath{0.27}}$
$\newcommand{\axisplanetcmaxLn}{\ensuremath{0.806}}$
$\newcommand{\RVoffFEROSmaxLn}{\ensuremath{51324.5}}$
$\newcommand{\RVoffHARPSmaxLn}{\ensuremath{51349.4}}$
$\newcommand{\RVoffCHIRONmaxLn}{\ensuremath{-0.2}}$
$\newcommand{\RVoffCORALIEmaxLn}{\ensuremath{51327.1}}$
$\newcommand{\RVjittFEROSmaxLn}{\ensuremath{21.1}}$
$\newcommand{\RVjittHARPSmaxLn}{\ensuremath{5.1}}$
$\newcommand{\RVjittCHIRONmaxLn}{\ensuremath{14.9}}$
$\newcommand{\RVjittCORALIEmaxLn}{\ensuremath{4.3}}$
$\newcommand{\julietpb}{\ensuremath{0.1015\pm{0.0005}}}$
$\newcommand{\julietbb}{\ensuremath{0.45_{-0.03}^{+0.02}}}$
$\newcommand{\julietrho}{\ensuremath{3431_{-140}^{+160}}}$
$\newcommand{\julietqoneTESS}{\ensuremath{0.30\pm0.03}}$
$\newcommand{\julietqtwoTESS}{\ensuremath{0.36_{-0.06}^{+0.05}}}$
$\newcommand{\julietqoneASTEP}{\ensuremath{0.62\pm0.06}}$
$\newcommand{\julietqtwoASTEP}{\ensuremath{0.07\pm0.04}}$
$\newcommand{\julietqoneLCOone}{\ensuremath{0.32_{-0.07}^{+0.08}}}$
$\newcommand{\julietqtwoLCOone}{\ensuremath{0.30\pm0.07}}$
$\newcommand{\julietqoneLCOtwo}{\ensuremath{0.61_{-0.06}^{+0.08}}}$
$\newcommand{\julietqtwoLCOtwo}{\ensuremath{0.43_{-0.06}^{+0.07}}}$
$\newcommand{\julietqoneLCOthr}{\ensuremath{0.29_{-0.06}^{+0.07}}}$
$\newcommand{\julietqtwoLCOthr}{\ensuremath{0.22_{-0.06}^{+0.07}}}$
$\newcommand{\julietqoneLCOfour}{\ensuremath{0.33_{-0.06}^{+0.07}}}$
$\newcommand{\julietqtwoLCOfour}{\ensuremath{0.25_{-0.06}^{+0.07}}}$
$\newcommand{\julietqoneLCOfive}{\ensuremath{0.71\pm0.07}}$
$\newcommand{\julietqtwoLCOfive}{\ensuremath{0.43\pm0.07}}$
$\newcommand{\julietqoneLCOsix}{\ensuremath{0.29_{-0.08}^{+0.07}}}$
$\newcommand{\julietqtwoLCOsix}{\ensuremath{0.25_{-0.05}^{+0.06}}}$
$\newcommand{\julietqonePEST}{\ensuremath{0.83\pm0.07}}$
$\newcommand{\julietqtwoPEST}{\ensuremath{0.49_{-0.07}^{+0.06}}}$
$\newcommand{\julietqoneNEOS}{\ensuremath{0.67_{-0.07}^{+0.06}}}$
$\newcommand{\julietqtwoNEOS}{\ensuremath{0.41\pm0.07}}$
$\newcommand{\julietqoneHwd}{\ensuremath{0.50_{-0.07}^{+0.08}}}$
$\newcommand{\julietqtwoHwd}{\ensuremath{0.35_{-0.07}^{+0.08}}}$
$\newcommand{\julietmfluxTESStwo}{\ensuremath{0.0002_{-0.0013}^{+0.0012}}}$
$\newcommand{\julietsigmaTESStwo}{\ensuremath{402\pm8}}$
$\newcommand{\julietmfluxTESSten}{\ensuremath{0.003\pm0.002}}$
$\newcommand{\julietsigmawTESSten}{\ensuremath{327\pm13}}$
$\newcommand{\julietmfluxTESSthir}{\ensuremath{-0.0007_{-0.0005}^{+0.0006}}}$
$\newcommand{\julietsigmawTESSthir}{\ensuremath{371_{-11}^{+10}}}$
$\newcommand{\julietmfluxTESStn}{\ensuremath{0.0010_{-0.0005}^{+0.0006}}}$
$\newcommand{\julietsigmawTESStn}{\ensuremath{641\pm9}}$
$\newcommand{\julietmfluxTESStt}{\ensuremath{0.006_{-0.006}^{+0.005}}}$
$\newcommand{\julietsigmawTESStt}{\ensuremath{419_{-7}^{+8}}}$
$\newcommand{\julietmfluxTESSts}{\ensuremath{0.0012\pm0.0008}}$
$\newcommand{\julietsigmawTESSts}{\ensuremath{604_{-9}^{+10}}}$
$\newcommand{\julietmfluxTESSst}{\ensuremath{-0.0003_{-0.0028}^{+0.0030}}}$
$\newcommand{\julietsigmawTESSst}{\ensuremath{493\pm8}}$
$\newcommand{\julietmfluxASTEPone}{\ensuremath{0.0004\pm0.0001}}$
$\newcommand{\julietsigmawASTEPone}{\ensuremath{960_{-25}^{+34}}}$
$\newcommand{\julietmfluxASTEPtwo}{\ensuremath{0.00027\pm0.00007}}$
$\newcommand{\julietsigmawASTEPtwo}{\ensuremath{999.1_{-0.6}^{+0.9}}}$
$\newcommand{\julietmfluxASTEPthr}{\ensuremath{-0.00088\pm0.00007}}$
$\newcommand{\julietsigmawASTEPthr}{\ensuremath{998_{-1}^{+2}}}$
$\newcommand{\julietmfluxPEST}{\ensuremath{0.00004_{-0.00009}^{+0.00008}}}$
$\newcommand{\julietsigmawPEST}{\ensuremath{996\pm3}}$
$\newcommand{\julietmfluxNEOS}{\ensuremath{-0.00002\pm0.00003}}$
$\newcommand{\julietsigmawNEOS}{\ensuremath{999.92_{-0.05}^{+0.08}}}$
$\newcommand{\julietmfluxHwd}{\ensuremath{-0.050\pm0.0001}}$
$\newcommand{\julietsigmawHwd}{\ensuremath{995_{-3}^{+5}}}$
$\newcommand{\julietmfluxLCOone}{\ensuremath{-0.000003_{-0.000071}^{+0.000075}}}$
$\newcommand{\julietsigmawLCOone}{\ensuremath{779_{-95}^{+91}}}$
$\newcommand{\julietmfluxLCOtwo}{\ensuremath{-0.0003\pm0.0002}}$
$\newcommand{\julietsigmawLCOtwo}{\ensuremath{989_{-7}^{+9}}}$
$\newcommand{\julietmfluxLCOthr}{\ensuremath{0.0003\pm0.0001}}$
$\newcommand{\julietsigmawLCOthr}{\ensuremath{994_{-4}^{+7}}}$
$\newcommand{\julietmfluxLCOfo}{\ensuremath{-0.0001\pm0.0001}}$
$\newcommand{\julietsigmawLCOfo}{\ensuremath{986_{-9}^{+14}}}$
$\newcommand{\julietmfluxLCOfi}{\ensuremath{-0.00003\pm0.00014}}$
$\newcommand{\julietsigmawLCOfi}{\ensuremath{990_{-7}^{+12}}}$
$\newcommand{\julietmfluxLCOsix}{\ensuremath{-0.0082\pm0.00006}}$
$\newcommand{\julietsigmawLCOsix}{\ensuremath{998_{-1}^{+2}}}$
$\newcommand{\julietGPsigmaTESStwo}{\ensuremath{0.0031_{-0.0007}^{+0.0005}}}$
$\newcommand{\julietGPrhoTESStwo}{\ensuremath{4.0_{-0.7}^{+0.5}}}$
$\newcommand{\julietGPsigmaTESSten}{\ensuremath{0.0086_{-0.0006}^{+0.0005}}}$
$\newcommand{\julietGPrhoTESSten}{\ensuremath{0.70_{-0.04}^{+0.03}}}$
$\newcommand{\julietGPsigmaTESSthir}{\ensuremath{0.0031\pm0.0002}}$
$\newcommand{\julietGPrhoTESSthir}{\ensuremath{0.71_{-0.04}^{+0.03}}}$
$\newcommand{\julietGPsigmaTESStn}{\ensuremath{0.0035\pm0.0002}}$
$\newcommand{\julietGPrhoTESStn}{\ensuremath{0.42\pm0.02}}$
$\newcommand{\julietGPsigmaTESStt}{\ensuremath{0.009_{-0.004}^{+0.002}}}$
$\newcommand{\julietGPrhoTESStt}{\ensuremath{9_{-3}^{+2}}}$
$\newcommand{\julietGPsigmaTESSts}{\ensuremath{0.0054\pm0.0003}}$
$\newcommand{\julietGPrhoTESSts}{\ensuremath{0.42\pm0.02}}$
$\newcommand{\julietGPsigmaTESSst}{\ensuremath{0.007_{-0.002}^{+0.001}}}$
$\newcommand{\julietGPrhoTESSst}{\ensuremath{4.2_{-0.8}^{+0.6}}}$
$\newcommand{\julietTTESStwo}{\ensuremath{2458361.0283\pm0.0008}}$
$\newcommand{\julietTTESSten}{\ensuremath{2458570.732\pm0.001}}$
$\newcommand{\julietTTESSthir}{\ensuremath{2458675.6182\pm0.0009}}$
$\newcommand{\julietTTESStn}{\ensuremath{2459095.1087_{-0.0009}^{+0.0010}}}$
$\newcommand{\julietTTESStt}{\ensuremath{2459200.0050\pm0.0006}}$
$\newcommand{\julietTTESSts}{\ensuremath{2459304.8770_{-0.0010}^{+0.0009}}}$
$\newcommand{\julietTTESSst}{\ensuremath{2460038.9735\pm0.0004}}$
$\newcommand{\julietTASTEPone}{\ensuremath{2459304.8774\pm0.0008}}$
$\newcommand{\julietTASTEPtwo}{\ensuremath{2459409.7229\pm0.0007}}$
$\newcommand{\julietTASTEPthr}{\ensuremath{2459829.2283\pm0.0005}}$
$\newcommand{\julietTLCOone}{\ensuremath{2459200.04\pm0.03}}$
$\newcommand{\julietTLCOtwo}{\ensuremath{2459619.482\pm0.002}}$
$\newcommand{\julietTLCOthr}{\ensuremath{2459619.480\pm0.001}}$
$\newcommand{\julietTLCOfo}{\ensuremath{2459619.486\pm0.001}}$
$\newcommand{\julietTLCOfi}{\ensuremath{2459619.485\pm0.001}}$
$\newcommand{\julietTLCOsix}{\ensuremath{2459934.0818\pm0.0003}}$
$\newcommand{\julietTPEST}{\ensuremath{2459200.019\pm0.001}}$
$\newcommand{\julietTNEOS}{\ensuremath{2459304.900_{-0.003}^{+0.002}}}$
$\newcommand{\julietTHwd}{\ensuremath{2459304.759\pm0.002}}$
$\newcommand{\julietP}{\ensuremath{104.87236\pm0.00005}}$
$\newcommand{\julietaRs}{\ensuremath{126\pm2}}$
$\newcommand{\juliettzero}{\ensuremath{2458256.1286\pm0.0006}}$
$\newcommand{\julieta}{\ensuremath{0.480_{-0.008}^{+0.007}}}$
$\newcommand{\julietRp}{\ensuremath{0.810\pm0.005}}$</div>



<div id="title">

# TOI-199 b: A well-characterized 100-day transiting warm giant planet with TTVs seen from Antarctica

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.14915-b31b1b.svg)](https://arxiv.org/abs/2309.14915)<mark>Appeared on: 2023-09-27</mark> -  _33 pages, 23 figures. Accepted for publication in AJ_

</div>
<div id="authors">

M. J. Hobson, et al. -- incl., <mark>T. Trifonov</mark>, <mark>J. Eberhardt</mark>

</div>
<div id="abstract">

**Abstract:** We present the spectroscopic confirmation and precise mass measurement of the warm giant planet TOI-199 b. This planet was first identified in _TESS_ photometry and confirmed using ground-based photometry from ASTEP in Antarctica including a full 6.5 h long transit, PEST, Hazelwood, and LCO; space photometry from NEOSSat; and radial velocities (RVs) from FEROS, HARPS, CORALIE, and CHIRON. Orbiting a late G-type star, TOI-199 b has a $\mathrm{\periodplanetb   d}$ period, a mass of $\mathrm{\massplanetb   M_J}$ , and a radius of $\mathrm{\julietRp   R_J}$ . It is the first warm exo-Saturn with a precisely determined mass and radius. The _TESS_ and ASTEP transits show strong transit timing variations, pointing to the existence of a second planet in the system. The joint analysis of the RVs and TTVs provides a unique solution for the non-transiting companion TOI-199 c, which has a period of $\mathrm{\periodplanetc   d}$ and an estimated mass of $\mathrm{\massplanetc   M_J}$ . This period places it within the conservative Habitable Zone.

</div>

<div id="div_fig1">

<img src="tmp_2309.14915/./ground_based_photometry_times.png" alt="Fig9" width="100%"/>

**Figure 9. -** Follow-up photometry for TOI-199. Top row: Light curves for ASTEP observations - an egress on 31st March 2021 (left), a full transit on 13th July 2021 (centre), and a full transit on 6th September 2022 (right). Middle Row: Light curves for PEST (left, egress on 16th December 2020), NEOSSat (centre, full transit on 31st March 2021), and Hazelwood (right, egress on 31st March 2021) observations. Bottom row: light curves for LCO observations - a pre-ingress flat curve on 16th December 2020 (left), two ingresses and egresses on 8th February 2022 (centre), for which the points are colour-coded by filter and site, and a full transit on 20th December 2022 (right). In all panels, the dashed vertical line indicates the transit midpoint, and the dotted vertical lines the egress and ingress. (*fig:followup-lc*)

</div>
<div id="div_fig2">

<img src="tmp_2309.14915/./TOI-199_rvs_all_fit10.png" alt="Fig14.1" width="25%"/><img src="tmp_2309.14915/./TOI-199_ttvs.png" alt="Fig14.2" width="25%"/><img src="tmp_2309.14915/./TOI-199_phasefold_1_fit10_bin2.png" alt="Fig14.3" width="25%"/><img src="tmp_2309.14915/./TOI-199_phasefold_2_fit10_bin2.png" alt="Fig14.4" width="25%"/>

**Figure 14. -** * Top*: Radial velocity measurements (FEROS: blue, HARPS: red, CHIRON: purple, CORALIE: green) and best-fit two-planet model (grey line) for the combined data. * Bottom left*: TTVs (blue circles) and fitted model with \texttt{Exo-Striker}(grey line) for TOI-199 b (top panel) and residuals to the model (bottom panel). The model has been smoothed with a quadratic spline. * Bottom centre and right*:  phase-folded representation of the two planetary signals after the RV signal of the other companion was subtracted. The respective RV residuals are shown under each panel, accordingly. The more precise HARPS RVs, which are the main driver of the fit, are highlighted. The black squares show the binned phase-folded RVs. (*fig:RVs_TTVs*)

</div>
<div id="div_fig3">

<img src="tmp_2309.14915/./comparison_lightcurves_zoom_4.png" alt="Fig8" width="100%"/>

**Figure 8. -** _TESS_ light curves for TOI-199, for the seven sectors with transits. The short-cadence PDCSAP light curves are shown in blue, and the FFI light curves extracted with \texttt{tesseract} in orange. For Sectors 10 and 32 the transits could not be correctly recovered from the PDCSAP light curves. (*fig:lightcurve*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.14915"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

279  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
